# Building a flirtation-detection language model

Can a computer learn to detect flirting? Maybe!

Using Soma's homework and replacing it with my own data, I tried to create a model using authentic Tinder messages to detect flirtation. 

## The data

How does one find Tinder data? By emailing benevolent strangers.

Special thanks to Kris from [swipestats.io]("swipestats.io") for sharing the Tinder data with me. Quite literally, I couldn't have done it without you. <br>
Kristian Elset Bø <br>
email: kristian@boe.ventures <br>
LinkedIn: https://www.linkedin.com/in/kristianeboe/


### Context and importing datasets
I have over half a million lines of Tinder messages. This was retrieved after hours of cleaning.

Due to lack of time and experience, I used only ~5800 total messages. The messages selected contain a 50/50 mix of male and female users.

The training data contains ~2800 unique messages and is labeled *flirting_rated.csv*. The testing data also contains ~3000 unique messages and is labeled *test_cv.csv*. 

The training data contains two columns— polarity and final_messages. The polarity was manually rated by humans (Thank you, Carson) and ranges between 0 = no flirtation and 1 = flirtation. The original dataset contained 3000 messages but rows with personal information was removed to protect privacy.

The testing data does not contain a polarity score.


In [3]:
import pandas as pd
pd.set_option("display.max_colwidth", 200)
pd.set_option("display.max_columns", 200)

from google.colab import files
uploaded = files.upload()

df = pd.read_csv("flirting_rated.csv")
test = pd.read_csv("test_cv.csv")

# Extra cleaning
df = df.dropna()

Saving test_cv.csv to test_cv.csv
Saving flirting_rated.csv to flirting_rated (2).csv


## Training the model

To build my model, I used the machine learning library called [scikit-learn](https://scikit-learn.org/stable/). 

In [8]:
! pip install numpy
! pip install sklearn

### Counting words

Creating a **bag of words model** using `TfidfVectorizer`.


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
vectorizer = TfidfVectorizer(max_features=4000)
vectors = vectorizer.fit_transform(df.final_messages)
words_df = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names())
words_df.head()

,05,06,10,11,12,13,135,13th,14,15,150,16,17,18,19,1989,1am,1st,20,2003,2013,2032,2034,21,21st,2200,23,2500,29,2nd,2x,30,36,3rd,40,40k,45,4loko,4th,50,500,5000,5th,60,65,70,7s,80,800,90,90s,94,ab,abdominal,abetting,able,about,above,absolutely,absurd,absurdly,acab,accent,access,accidentally,accidetanlly,accommodate,accompanied,according,account,accounting,accounts,achieved,acid,acoustic,acquiring,across,act,activities,activity,actual,actually,add,added,addicted,addy,admin,admit,adopted,advanced,adventures,advertised,advertising,affected,after,afterwards,again,against,age,ago,...,wht,why,wicked,wide,wifi,wig,wii,wild,wildlife,will,willing,wilson,win,window,wine,wings,wins,winter,wire,wisconsin,wish,witch,with,without,woes,woke,wolfhound,woman,wombats,women,won,wondered,wonderful,wondering,wood,wooden,word,wordplay,words,work,worked,working,works,world,worried,worries,worry,worse,worst,worth,would,wouldn,wounds,wow,wowwwww,wpm,wrangling,wrapped,wrestle,wrestling,wretched,write,writing,written,wrong,wrote,wtf,xcx,ya,yah,yanny,yeah,year,years,yellow,yep,yes,yesterday,yet,yikes,yo,yodel,yogurt,york,you,youd,young,younger,your,yours,yourself,youtube,youuu,yuck,yuh,yup,zappa,zero,zinger,zoom
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.416476,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [83]:
#vectorizer.get_feature_names()

### Setting up the variables and training a language model

`X` is the **features**, the things we use to predict flirting or not flirting. 

`y` is the **labels**, the flirting or not flirting rating that we want to predict.

In [54]:
X = words_df
y = df.polarity

### Selecting an architecture

Why did I choose these architectures? I have no idea!

Soma included this in our homework and after tinkering with the data, I decided these give me the best results. Do I know the difference? Not really, but I'm still learning. 

In [55]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

### Training the language models

In [56]:
%%time
# Create and train a random forest classifier
forest = RandomForestClassifier(n_estimators=50)
forest.fit(X, y)

CPU times: user 6.37 s, sys: 0 ns, total: 6.37 s
Wall time: 6.34 s


In [57]:
%%time
# Create and train a linear support vector classifier (LinearSVC)
svc = LinearSVC()
svc.fit(X, y)

CPU times: user 63.4 ms, sys: 0 ns, total: 63.4 ms
Wall time: 64.2 ms


In [58]:
%%time
# Create and train a multinomial naive bayes classifier (MultinomialNB)
bayes = MultinomialNB()
bayes.fit(X, y)

CPU times: user 74.1 ms, sys: 0 ns, total: 74.1 ms
Wall time: 50.5 ms


## Using the models and preparing the data

The testing data is used for this portion. 

I converted the training data into a list before transferring it to a dataframe. Could I have done this portion in a easier method? Probably. 

In [60]:
# Converting to list
convos = test['final_messages'].tolist()

unknown = pd.DataFrame({'content': convos})
unknown = unknown.dropna()
unknown

,content
0,That was so
1,Fucking
2,Loud
3,And they never fixed it
4,My room was like the epicenter
...,...
2995,doges
2996,i didn't spell it wrong
2997,it's a meme
2998,seems like you don't have a sense of humour


### Vectorizing testing data

Computers like to read things in their own complicated way. It works for them, but not for humans. I vectorized the text using the same words it learned earlier.

In [63]:
# Put it through the vectorizer
unknown_vectors = vectorizer.transform(unknown.content)
unknown_words_df = pd.DataFrame(unknown_vectors.toarray(), columns=vectorizer.get_feature_names())
unknown_words_df.head()

,05,06,10,11,12,13,135,13th,14,15,150,16,17,18,19,1989,1am,1st,20,2003,2013,2032,2034,21,21st,2200,23,2500,29,2nd,2x,30,36,3rd,40,40k,45,4loko,4th,50,500,5000,5th,60,65,70,7s,80,800,90,90s,94,ab,abdominal,abetting,able,about,above,absolutely,absurd,absurdly,acab,accent,access,accidentally,accidetanlly,accommodate,accompanied,according,account,accounting,accounts,achieved,acid,acoustic,acquiring,across,act,activities,activity,actual,actually,add,added,addicted,addy,admin,admit,adopted,advanced,adventures,advertised,advertising,affected,after,afterwards,again,against,age,ago,...,wht,why,wicked,wide,wifi,wig,wii,wild,wildlife,will,willing,wilson,win,window,wine,wings,wins,winter,wire,wisconsin,wish,witch,with,without,woes,woke,wolfhound,woman,wombats,women,won,wondered,wonderful,wondering,wood,wooden,word,wordplay,words,work,worked,working,works,world,worried,worries,worry,worse,worst,worth,would,wouldn,wounds,wow,wowwwww,wpm,wrangling,wrapped,wrestle,wrestling,wretched,write,writing,written,wrong,wrote,wtf,xcx,ya,yah,yanny,yeah,year,years,yellow,yep,yes,yesterday,yet,yikes,yo,yodel,yogurt,york,you,youd,young,younger,your,yours,yourself,youtube,youuu,yuck,yuh,yup,zappa,zero,zinger,zoom
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

### Predicting with our models

Now we can try making predictions! Did it work? 

In [64]:
# Predict using all our models. 

# Random forest predictions + probabilities
unknown['pred_forest'] = forest.predict(unknown_words_df)
unknown['pred_forest_proba'] = forest.predict_proba(unknown_words_df)[:,1]

# SVC predictions (doesn't support probabilities)
unknown['pred_svc'] = svc.predict(unknown_words_df)

# Bayes predictions + probabilities
unknown['pred_bayes'] = bayes.predict(unknown_words_df)
unknown['pred_bayes_proba'] = bayes.predict_proba(unknown_words_df)[:,1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [86]:
unknown.head(10)

,content,pred_forest,pred_forest_proba,pred_svc,pred_bayes,pred_bayes_proba
0,That was so,0.0,0.105000,0.0,0.0,0.027569
1,Fucking,1.0,0.623671,0.0,0.0,0.329309
2,Loud,1.0,0.544026,0.0,0.0,0.175078
3,And they never fixed it,0.0,0.000000,0.0,0.0,0.053155
4,My room was like the epicenter,0.0,0.200000,0.0,0.0,0.082934
5,It was that bad,0.0,0.020000,0.0,0.0,0.025459
6,And of course the time it gets bad is when it cools down at night,0.0,0.040000,0.0,0.0,0.013075
7,Moved from hegeman,0.0,0.197503,0.0,0.0,0.063920
8,Hahahahahahahahha,1.0,0.544026,0.0,0.0,0.175078
9,Other than that perkins was dope,0.0,0.060000,0.0,0.0,0.075418


It could be better.

## Testing our models



Instead of reading all 3000 messages from the testing data to determine accuracy, I'll use **train-test split** and a **confusion matrix**.


In [67]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [68]:
print("Training random forest")
forest.fit(X_train, y_train)

print("Training SVC")
svc.fit(X_train, y_train)

print("Training Naive Bayes")
bayes.fit(X_train, y_train)

Training random forest
Training SVC
Training Naive Bayes


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

### Confusion matrices


In [69]:
from sklearn.metrics import confusion_matrix

#### Applying the confusion matrix

In [70]:
# Random forest
y_true = y_test
y_pred = forest.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['non-flirting', 'flirting'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names).div(matrix.sum(axis=1), axis=0)

,Predicted non-flirting,Predicted flirting
Is non-flirting,0.910169,0.089831
Is flirting,0.542636,0.457364


In [71]:
forest.score(X_test, y_test)

0.8289290681502086

In [72]:
# Linear SVC
y_true = y_test
y_pred = svc.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['non-flirting', 'flirting'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names).div(matrix.sum(axis=1), axis=0)

,Predicted non-flirting,Predicted flirting
Is non-flirting,0.967797,0.032203
Is flirting,0.604651,0.395349


In [73]:
svc.score(X_test, y_test)

0.8650904033379694

In [74]:
# Multinomial Naive Baynes
y_true = y_test
y_pred = bayes.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['non-flirting', 'flirting'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names).div(matrix.sum(axis=1), axis=0)

,Predicted non-flirting,Predicted flirting
Is non-flirting,0.996610,0.003390
Is flirting,0.790698,0.209302


In [75]:
bayes.score(X_test, y_test)

0.8553546592489569

In [89]:
df.polarity.value_counts()

0.0    2370
1.0     503
Name: polarity, dtype: int64

# Final thoughts 

I'll be the first to admit that this flirtation model is rudimentary at best. I'll also admit that just about everything displayed comes straight from Soma's homework. 

There are several flaws to this model and I'll categorize them as 1. Data Selection, 2. Size, 3. Consistency. 

1. Data Selection
> When I selected the data, I did not do it randomly. Instead, I selected the first 1500 and merged it with the last 1500 messages. Why? Because the first half of the whole conversations data were only male-user whereas the last half was only female-users. It has to do with how I merged the dataframes. Additionally, this meant that the messages used in the testing and training data came from only a handful of users. For next time, I'll take the time to find a function that will randomly select the messages used.
2. Size
> My model needs much more examples to train on. Out of the ~3000 messages, only 503 were rated as 1. 
3. Consistency
> When I was rating the polarity of the training dataset, I realized that I did't have a proper definition of what flirting means. What Carson (Thank you again) thought was "not flirting", I would consider otherwise and vice versa. Does that mean I should include more fields to measure? More than likely.

All in all, while it was fun to see what the flirtation model could look like but I have a long journey ahead of me. There's so much more I need to learn when it comes to sentiment analysis, NLP, and general machine learning. I find it to be excellent practice for the long run and am eager to see what other projects I can create next. 